# ZSim Buff系统重构架构设计书 V 0.2

**版本**: 0.2 
**状态**: 已落地 / 测试中  
**对应代码分支**: `zsim-korlne`

## 1. 概述

本设计书基于实际落地的代码结构编写。与最初的《Buff重构方案》相比，当前架构在事件分发机制上进行了优化：不再构建一个显式的、集中式的 `EventRouter` 类，而是采用了**基于注册表的分布式 Handler 模式**。Buff系统彻底解耦为“状态容器”、“生命周期管理”、“效果定义”与“事件监听”四个维度。

### 核心设计理念

1.  **Buff即容器 (State Container)**: `Buff` 对象不再包含任何业务逻辑（如判定、计算），只负责维护层数、时间等状态。
2.  **工厂模式 (Factory)**: 所有 Buff 实例由 `GlobalBuffController` 统一生产，并在生产时注入效果。
3.  **效果分离 (Effect Segregation)**: Buff 的“数值被动”与“机制触发”被拆分为 `BonusEffect` 和 `TriggerEffect`，分别走不同的生效路径。
4.  **去中心化事件监听 (Distributed Listening)**: 通过 `BuffTriggerHandler` 动态注册到全局事件系统，替代了原定的大型事件路由树。

## 2. 系统整体架构图

系统主要分为三层：**控制层 (Controller)**、**管理层 (Manager)** 和 **执行/实体层 (Entity/Handler)**。

```mermaid
classDiagram
    %% 全局控制层
    class GlobalBuffController {
        +instantiate_buff(buff_id) Buff
        -_trigger_db: DataFrame
        -_effect_db: Dict
        -_create_effects_for_buff()
    }

    %% 角色级管理层
    class BuffManager {
        +active_buffs: Dict
        +add_buff()
        +remove_buff()
        +tick()
    }

    %% 实体层
    class Buff {
        +ft: BuffFeature (Static)
        +dy: BuffDynamic (Runtime)
        +effects: List[EffectBase]
        +start()
        +end()
        +refresh()
    }

    %% 效果层
    class EffectBase {
        <<Abstract>>
    }
    class BonusEffect {
        +target_attribute: str
        +value: float
    }
    class TriggerEffect {
        +trigger_event_type: str
        +callback_logic_id: str
    }

    %% 事件系统对接
    class BuffTriggerHandler {
        +supports(event) bool
        +handle(event)
    }
    class BonusPool {
        +query_total_add()
    }

    GlobalBuffController ..> Buff : Creates
    GlobalBuffController ..> EffectBase : Creates
    BuffManager o-- Buff : Manages
    Buff *-- EffectBase : Contains
    EffectBase <|-- BonusEffect
    EffectBase <|-- TriggerEffect
    BonusEffect --o BonusPool : Syncs to
    TriggerEffect --o BuffTriggerHandler : Generates
```

## 3. 关键模块详解

### 3.1 GlobalBuffController (工厂与数据源)

- **职责**: 单例对象，负责加载 `触发判断.csv` 和 `buff_effect.csv`。它是系统中唯一有权创建 `Buff` 实例的地方。
- **核心逻辑 (`instantiate_buff`)**:
    1. 根据 ID 查表获取配置。
    2. 创建“干净”的 Buff 对象。
    3. 解析效果表，创建 `BonusEffect` 列表。
    4. 解析触发配置，创建 `TriggerEffect` 列表。
    5. 将所有 Effect 注入 Buff 对象并返回。

### 3.2 Buff (纯状态实体)

严格遵循 SRP (单一职责原则)，移除了所有判定逻辑。
- **BuffFeature (`.ft`)**: 只读，存储静态配置（如最大层数、是否是Debuff）。
- **BuffDynamic (`.dy`)**: 读写，存储运行时状态（当前层数、结束tick、自定义数据 `custom_data`）。
- **方法**: 仅包含 `start`, `end`, `refresh`, `add_stack` 等对 `.dy` 进行操作的方法。

### 3.3 Effect System (双轨制效果)

Buff 的效果被明确分为两类，并在实例化时确定：

1.  **BonusEffect (数值型)**:
    - **描述**: 直接修改角色面板（如 攻击力+10%）。
    - **流向**: 当 Buff 被 Manager 添加时，这类 Effect 会被提取并注册到角色的 `BonusPool` 中。

2.  **TriggerEffect (机制型)**:
    - **描述**: 监听特定事件并执行逻辑（如 暴击时造成额外伤害）。
    - **流向**: 包含 `logic_id`（回调逻辑索引）和 `trigger_event_type`（监听事件类型）。当 Buff 激活时，Manager 会为其创建对应的 `BuffTriggerHandler`。

### 3.4 Event System Integration (替代EventRouter)

原计划中的 `EventRouter` 被现有的 `ZSimEventSystem` + `Handler Registry` 替代。这是最大的架构变更点。

- **BuffTriggerHandler**:
    - 每个带有触发机制的 Buff 都会生成一个 Handler 实例。
    - **`supports(event)`**: 负责过滤事件。检查条件包括：
        - Buff 是否 Active。
        - EventType 是否匹配。
        - SourceID 是否为 Buff 持有者。
        - CD 是否转好。
        - Skill Tags 是否匹配。
    - **`handle(event)`**: 从 `BuffCallbackRepository` 中根据 `logic_id` 获取 Python 函数并执行。

## 4. 核心业务流程

### 4.1 Buff 的创建与应用流程

此流程展示了如何从 ID 变为生效的属性加成。

```mermaid
sequenceDiagram
    participant Sim as Simulator/Skill
    participant Mgr as BuffManager
    participant GBC as GlobalBuffController
    participant Buff as BuffInstance
    participant Pool as BonusPool
    participant Reg as EventRegistry

    Sim->>Mgr: add_buff("1001_ATK_UP")
    
    alt Buff不存在
        Mgr->>GBC: instantiate_buff("1001_ATK_UP")
        GBC->>GBC: Load CSV Config
        GBC->>GBC: Create Effects (Bonus & Trigger)
        GBC-->>Mgr: 返回 Buff 对象 (含 Effects)
    end

    Mgr->>Buff: start() / add_stack()
    Buff->>Buff: 更新 dy.active = True

    par 处理数值效果
        Mgr->>Buff: 获取 BonusEffects
        Mgr->>Pool: register_effects(BonusEffects)
        Pool->>Pool: 重新计算属性面板
    and 处理触发效果
        Mgr->>Buff: 获取 TriggerEffects
        loop 对每个 TriggerEffect
            Mgr->>Reg: 注册 BuffTriggerHandler(buff, effect)
        end
    end
```

### 4.2 Buff 的事件触发流程

此流程展示了“机制型 Buff”如何响应系统事件（替代了EventRouter逻辑）。

```mermaid
sequenceDiagram
    participant Event as ZSimEvent
    participant Sys as EventHandlerRegistry
    participant Hdl as BuffTriggerHandler
    participant Buff as BuffInstance
    participant Logic as CallbackRepository

    Note over Event: 例如: SkillHitEvent 产生
    Event->>Sys: publish(event)
    Sys->>Sys: 遍历所有注册的 Handler
    
    Sys->>Hdl: supports(event)?
    Hdl->>Buff: check dy.active
    Hdl->>Hdl: check EventType, Tags, CD
    
    alt check pass
        Hdl-->>Sys: True
        Sys->>Hdl: handle(event)
        Hdl->>Logic: get_callback(logic_id)
        Logic->>Logic: 执行具体业务逻辑
        Logic-->>Hdl: 返回新事件 (可选)
        Hdl->>Buff: update last_trigger_tick (Update CD)
    else check fail
        Hdl-->>Sys: False
    end
```

## 5. 数据结构定义

### 5.1 数据库结构 (`_trigger_db`)
从 `触发判断.csv` 加载，包含 Buff 的元数据。

| 字段名 | 类型 | 描述 |
| :--- | :--- | :--- |
| BuffName | str | 唯一索引 ID |
| maxduration | int | 默认持续时间 (tick) |
| maxcount | int | 最大层数 |
| logic_id | str | 触发逻辑的回调函数ID (关联代码) |
| trigger_type | str | 监听的事件类型 (如 `skill_hit`) |
| skill_tags | list | 触发所需的技能标签 |

### 5.2 效果数据结构 (`_effect_db`)
从 `buff_effect.csv` 加载，转换为字典缓存。

```python
{
    "Buff_ID_1": {
        "攻击力%": 0.1,    # BonusEffect
        "冰属性伤害%": 0.15 # BonusEffect
    },
    // 触发器类Buff在此表可能为空，或仅包含静态描述
}
```

## 6. 总结：当前架构 vs 原计划

| 关注点 | 原计划 (Buff重构方案.md) | 当前落地架构 |
| :--- | :--- | :--- |
| **核心中枢** | `EventRouter` (负责解析、分发、维护事件树) | `ZSimEventHandlerRegistry` (通用事件注册表) |
| **Buff触发** | Buff注册到 `event_trigger_tree` 的节点上 | Buff生成独立的 `BuffTriggerHandler` 注册到系统 |
| **判定逻辑** | 事件画像 (`EventProfile`) 提取信息供节点判断 | Handler 的 `supports()` 方法内部自包含判断逻辑 |
| **ID类型** | 计划引入 `int` ID | 沿用 `str` (BuffName) 作为主键，便于调试和CSV对应 |
| **优势** | 结构化强，适合复杂树状依赖 | **解耦性更强**，利用现有的Python装饰器和类继承特性，开发新Buff只需写Handler逻辑，无需维护全局树 |

当前架构成功实现了Buff逻辑的去中心化，极大地降低了新增Buff时的代码侵入性。开发者只需关注 CSV 配置和对应的 Callback 函数实现，无需关心底层事件分发细节。